<a href="https://colab.research.google.com/github/CanKeles5/ObjectDetection/blob/main/Stanford_Dataset_to_YOLO_Format_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2

In [3]:
#create the dataset, train, val folders
path_dataset = "/content/dataset"
path_images = path_dataset + "/images"
path_labels = path_dataset + "/labels"

path_train = path_dataset + "/train"
path_val = path_dataset + "/val"

os.mkdir(path_dataset)
os.mkdir(path_images)
os.mkdir(path_labels)
os.mkdir(path_images + "/train")
os.mkdir(path_images + "/val")
os.mkdir(path_labels + "/train")
os.mkdir(path_labels + "/val")

In [4]:
dataset_size = 12000

In [5]:
split = 5

In [6]:
dataset_path = "/content/drive/MyDrive/data-yolov5"

In [7]:
video_count = 0

For every video in videos, open the corresponding annotations file

In [8]:
for dirpath, dnames, fnames in os.walk(dataset_path + "/videos"):
    #print(str(dirpath) + " , " + str(dnames) + " , " + str(fnames))
    for f in fnames:
        if f == 'video.mov':
          video_count += 1

In [9]:
print("Number of videos is: " + str(video_count))

Number of videos is: 33


**dataset_size/video_count** is the number of frames we will extract from each video

In [10]:
num_frames = int(dataset_size/video_count)

In [11]:
print(num_frames)

363


Extract frames

In [ ]:
data_count = 0

for dirpath, dnames, fnames in os.walk(dataset_path + "/videos"):
    for f in fnames:
        if f == 'video.mov':
          video_path = dirpath + "/video.mov"
          annot_path = dirpath.replace("videos", "annotations") + "/annotations.txt"

          print("Processing video: " + video_path)

          ##### Process the video #####
          vidcap = cv2.VideoCapture(video_path)
          success, image = vidcap.read()

          frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

          count = 0

          while success:
            if count % int(frame_count/num_frames) == 0:
              
              write_to = ""
              if data_count % split == 0:
                write_image_to = "/content/dataset/images/val"
                new_annot = open("/content/dataset/labels/val/frame" + str(data_count) + ".txt", "x")
              else:
                write_image_to = "/content/dataset/images/train"
                new_annot = open("/content/dataset/labels/train/frame" + str(data_count) + ".txt", "x")
              
              cv2.imwrite(write_image_to + "/frame" + str(data_count) + ".jpg", image)

              #### Create annotation
              annot_file = open(annot_path, "r")
              
              #Read the annotation line by line
              while True:
                line = annot_file.readline()

                if not line:
                  break

                _, x_min, y_min, x_max, y_max, frameNo, lost, _, _, className = line.split()

                frameNo = int(frameNo)
                
                if frameNo==count:
                  x_min = float(x_min)
                  y_min = float(y_min)
                  x_max = float(x_max)
                  y_max = float(y_max)
                  lost = int(lost)
                  
                  className = className.replace('"', "")

                  classNo = -1

                  if className=="Biker":
                    classNo = 0
                  elif className=="Pedestrian":
                    classNo = 1
                  elif className=="Skater":
                    classNo = 2
                  elif className=="Cart":
                    classNo = 3
                  elif className=="Car":
                    classNo = 4
                  elif className=="Bus":
                    classNo = 5

                  if lost==0:
                    img_height, img_width, _ = image.shape

                    bbox_width = x_max - x_min
                    bbox_height = y_max - y_min

                    m_x = (bbox_width / 2 + x_min)/img_width
                    m_y = (bbox_height / 2 + y_min)/img_height

                    bbox_width = (x_max - x_min)/img_width
                    bbox_height = (y_max - y_min)/img_height


                    new_annot.write(str(classNo) + " " + str(m_x) + " " + str(m_y) + " " + str(bbox_width) + " " + str(bbox_height) + "\n")
              
              new_annot.close()
              annot_file.close()

              data_count+=1
            
            success, image = vidcap.read()
            count += 1
          
          #############################


Create annotations for each frame

In [ ]:
import shutil
shutil.make_archive("dataset_1000", 'zip', "/content/dataset")

'/content/dataset_1000.zip'